In [1]:
import json
import re
import jsonlines
status = ""
OUTFILE = "./bigdata/persona_selection_session3_notrelatedsetting_postprocessed.jsonl"
with open("../bigdata/session3_selected_persona_notrelatedsetting.txt","r") as f1, jsonlines.open("../bigdata/extracted_persona_session3.jsonl","r") as f2, open(OUTFILE,"w") as outfile:

    num = -1
    
    persona_lists = []
    for dial in f2:
        persona_lists.append(dial['session1_persona']+dial['session2_persona'])
        
    
    dic = {}
    persona_already_added = False
    for line in f1:
        d_mode = re.search(r"Dialogue (\d+):", line)
        u_mode = re.search(r"Utterance (\d+):", line)
        p_mode = re.search(r"Selected Persona:", line)
        
        ismode = False
        if d_mode or u_mode or p_mode:
            ismode = True
        
        if status=="Dialogue" and not ismode:
            pass
            
        elif status=="Utterance" and not ismode:
            dic["current"].append(line)
        elif status=="Selected Persona" and not ismode and not persona_already_added:
            persona_match = re.search(r"P(\d+)-Speaker [12]",line)
            
            if persona_match:
                if int(persona_match.group(1)) == 0:
                    dic["persona"].append("no persona")
                else:
                    dic["persona"].append(persona_lists[num][int(persona_match.group(1))-1])
                persona_already_added = True
            
                
        
        if d_mode:
            status="Dialogue"
            if len(dic.keys()) > 1:
                dic = {"num":num,**dic}
                if persona_already_added == False:
                    dic["persona"].append("no persona")
                try:
                    assert len(dic["current"]) == len(dic["persona"])
                except Exception as e:
                    print(num)
                    print("*"*10)
                    for i in dic["persona"]:
                        print(i)
                    print(len(dic["current"]), len(dic["persona"]))
                    assert False
                json.dump(dic,outfile)
                outfile.write('\n')
            dic={"current":[],"persona":[]}
            num+=1
        elif u_mode:
            if persona_already_added == False and len(dic["current"]) > 0:
                dic["persona"].append("no persona")
            status="Utterance"
        elif p_mode:
            persona_already_added = False
            status="Selected Persona"

            
            
        